In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torchvision.transforms as transforms

from torch.nn import Module
from torch.nn import Conv1d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import LogSoftmax
from torch import flatten

import pandas as pd

In [2]:
class CNN(Module):
    def __init__(self, numChannels=20):
        # call the parent constructor
        super(CNN, self).__init__()
        # initialize first set of CONV => RELU => POOL layers
        self.conv1 = Conv1d(in_channels=numChannels, out_channels=10,
            kernel_size=15, padding=7)
        self.relu1 = ReLU()
        # initialize second set of CONV => RELU => POOL layers
        self.conv2 = Conv1d(in_channels=10, out_channels=1,
            kernel_size=15, padding=7)
        self.relu2 = ReLU()
        # initialize our softmax classifier
        self.logSoftmax = LogSoftmax(dim=1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        output = self.logSoftmax(x)
        return output

In [3]:
class SequenceDataset(Dataset):
    def __init__(self, transform):
        self.data = pd.read_pickle('../data/cnn/data.csv')
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # x = input[index].T # channels should be first
        # y = output[index].reshape((1, -1)) # make the data match the shape of X after passing through all layers
        x = self.data['in'].iloc[index]
        # if index == 1:
        #     print(x.shape)
        # x = x.T
        y = self.data['out'].iloc[index]
        y = y.reshape((1,-1))

        if self.transform:
            x = self.transform(x)[0]

        return x, y

In [4]:
# input = np.random.uniform(low=0, high=1, size=(100, 100, 20))
# output = np.random.uniform(low=0, high=1, size=(100, 100))

In [5]:
dataset = SequenceDataset(transforms.ToTensor())
length = len(pd.read_pickle('../data/cnn/data.csv'))
train_len = (length * 9) // 10
test_len = length - train_len
train_set, test_set = torch.utils.data.random_split(dataset, [train_len, test_len])

train_loader = DataLoader(dataset=train_set, batch_size=1, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=1, shuffle=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = CNN().double()

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [6]:
for epoch in range(3):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader):
        inputs, target = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

(20, 146)
[1,  2000] loss: 0.017
[1,  4000] loss: 0.017


KeyboardInterrupt: 

In [125]:
df = pd.read_pickle('../data/cnn/data.csv')

dtype('int64')